# Cheatsheet Feature Engineering

## **1. Métodos**

## Imputación

✔️ Cuando hay datos faltantes

- Si los valores faltantes son inferiores al 5 % de la variable: media/mediana o el reemplazo de muestra aleatoria. 
    - Si es normal o quasi-normal: media, si no mediana
    - Imputar por categoría más frecuente si los valores faltantes son superiores al 5 % de la variable. 
    - Realizar la imputación de media/mediana + agregar una variable binaria adicional para capturar los valores faltantes y agregar una etiqueta "Faltante" en las variables categóricas.

- Si la cantidad de valores faltantes en una variable es pequeña: media/muestra aleatoria, para preservar la distribución de la variable.
- Si la variable/objetivo que se está tratando de predecir está muy desequilibrada: valores faltantes sea de hecho informativa.

- Si MNAR y no queremos atribuir la ocurrencia más común a NA, y si no queremos variable adicional para indicar la falta de datos:reemplazar por un valor en el extremo más alejado de la distribución o un valor arbitrario.

## Codificación

✔️ Para transformar varibles categóricas en números

1. **Codificación One-Hot (OHE):** modelos lineales `pd.get_dummies(<column>, drop_first=True)`

2. **Codificación ordinal:** reemplazar las etiquetas por algún número ordinal

3. **Codificación de recuento y frecuencia:** Cuando queremos capturar la importancia de la frecuencia de categoría (causa problemas si diferentes categorías tienen frecuencias similares).

4. **Codificación de objetivo/codificación de media:** introducir una característica fuerte en el modelo si la media objetivo es predictiva del resultado.

5. **Peso de la evidencia:** para codificar variables categóricas para su clasificación.

In [ ]:
# Métodos 3-5
X_to_map = X_train[<columna>].value_counts().to_dict() # frecuencia

X_to_map = X_train.groupby(['<col1>'])['target'].mean().to_dict() # media

df['WoE'] = np.log(df['<target>']/df['<prob_of_columns>']) # WoE
X_to_map = df['WoE'].to_dict()


# reemplazamos las etiquetas
X_train.X2 = X_train.X2.map(X_to_map)
X_test.X2 = X_test.X2.map(X_to_map)

## Transformación

✔️ Para ayudar a hacer una característica más normal (normalizar).

1. Transformación logarítmica: log(x)

2. Transformación recíproca: 1/x

3. Transformación de raíz cuadrada: sqrt(x)

4. Transformación exponencial: exp(x)

5. Transformación de Box-Cox

In [ ]:
df['<column_log>'] = np.log(df['<column>']) # tx log
df['column_boxcox'], param = stats.boxcox(df['<column>']) # tx box-cox


In [ ]:
# Histogramas + Q-Q para visualizar si la variable se distribuye normalmente

def diagnostic_plots(df, variable):
    # Función para graficar un histograma y un gráfico Q-Q
    # Uno al lado del otro, para una determinada variable
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[variable].hist()

    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=pylab)

    plt.show()
    
diagnostic_plots(df, '<column>')

### Estandarizar: 
✔️ Cuando las escalas de las características son muy distintas.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Discretización
✔️ Para discretizR valores. Por ejemplo, para mejorar el performance de modelos lineales o eliminar Outliers.

1. Discretización de igual ancho: `pd.cut(<column>, bins = <num>, labels = [<labels>])`
2. Discretización de igual frecuencia: `qcutpd.qcut(<column>, q = <num>, labels = [<labes>])`

3. Discretización de conocimiento del dominio

4. Discretización mediante árboles de decisión

## Valores atípicos

✔️ Para identificar y eliminar valores atípicos

1. **Eliminación de valores atípicos:** < 2%

2. **Tratamiento de valores atípicos como valores faltantes:** < 5%

3. **Discretización:** cuando lo requiera el modelo (modelos lineales).

4. **Codificación superior/inferior/cero:** 
    - Normal: 3*std
    - No Normal: IQR

In [ ]:
# encontrar valores atípicos

# Normal
upper_bound = data['<column>'].mean() + 3* data['<column>'].std()
lower_bound = data['<column>'].mean() - 3* data['<column>'].std()

# No notrmal
IQR = data['<column>'].quantile(0.75) - data['<column>'].quantile(0.25)
lower_bound = data['<column>'].quantile(0.25) - (IQR * 3)
upper_bound = data['<column>'].quantile(0.75) + (IQR * 3)

# Identifiación
outliers = df[(df['<column>'] < lower_bound) | (df['<column>'] > upper_bound)]

# Opción 1: Eliminar outliers
outliers_removed = df[(df['<column>'] >= lower_bound) &  (df['<column>'] <= upper_bound)].copy()

# Opción 2: Reemplazar outliers con los límites (winsorización)
df['<column>_winsorized'] = df['<column>'].copy()
df.loc[df['<column>'] < lower_bound, '<column>_winsorized'] = lower_bound
df.loc[df['<column>'] > upper_bound, '<column>_winsorized'] = upper_bound

## **2. Selección de características**

✔️ Cuando tenemos un número grande de características y queremos simplificar

### Eliminar características constantes o cuasi constantes
✔️ Eliminar siempre

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0) # seleccionar características con varianza mayor a 0 o el valor que se desee
sel.fit(X_train)

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

### Selección univariada

Funciones de evaluación:
- Para tareas de regresión: f_regression, mutual_info_regression

- Para tareas de clasificación: chi2, f_classif, mutual_info_classif

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2

# K-Best
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)

# Percentile
X_new = SelectPercentile(chi2, percentile=10).fit_transform(X, y)

### Matriz de correlación

- Las buenas variables están altamente correlacionadas con el objetivo.
- Las variables deben estar correlacionadas con el objetivo pero no correlacionadas entre sí.

In [ ]:
corr_matrix = df.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df_new = df.drop(df.columns[to_drop], axis=1)

### Métodos de envoltura

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs = SFS(RandomForestRegressor(), 
           k_features=10, 
           forward=False, # True para forward selection, False para backward selection
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=3)

sfs = sfs.fit(np.array(X_train), y_train)

X_train.columns[list(sfs1.k_feature_idx_)]

### Métodos integrados

In [ ]:
from sklearn.linear_model import Lasso

## Selección de características con Lasso
sel_ = SelectFromModel(Lasso(alpha=100))
sel_.fit(X_train, y_train)


selected_feat = X_train.columns[(sel_.get_support())]
print('Características totales: {}'.format((X_train.shape[1])))
print('Características seleccionadas: {}'.format(len(selected_feat)))
print('Características con coeficientes que se reducen a cero: {}'.format( np.sum(sel_.estimator_.coef_ == 0)))


## Para ver la importancia de las características
lasso = Lasso(alpha=0.1)  # Ajustar alpha según sea necesario
lasso.fit(X_train, y_train)

importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': np.abs(lasso.coef_)})
importance = importance.sort_values('Importance', ascending=False)


## Criterios de métodos para selección:

### Entrada numérica, salida numérica

- Este es un problema de modelado predictivo de regresión con variables de entrada numérica.

- Las técnicas más comunes son utilizar un **coeficiente de correlación**, como la de Pearson para una correlación lineal o métodos basados ​​en rango para una correlación no lineal.

- Las pruebas empleadas son:
    - Coeficiente de correlación de Pearson (lineal).
    - Coeficiente de rango de Spearman (no lineal)


### Entrada numérica, salida categórica

- Este es un problema de modelado predictivo de clasificación con variables de entrada numérica.

- Este podría ser el ejemplo más común de un problema de clasificación,

- Una vez más, las técnicas más comunes se basan en la **correlación**, aunque en este caso, deben tener en cuenta el **objetivo categórico**.

- Podemos emplear las siguientes pruebas:

    - Coeficiente de correlación ANOVA (lineal).
    - Coeficiente de rango de Kendall (no lineal).

- Kendall supone que la variable categórica es ordinal.


### Entrada categórica, salida numérica

- Este es un problema de modelado predictivo de regresión con variables de entrada categóricas.

- Este es un ejemplo extraño de un problema de regresión (no lo encontraremos a menudo).

- Podemos usar los **mismos métodos de "entrada numérica, salida categórica"** ​​(descritos anteriormente), pero **al revés**.




### Entrada categórica, salida categórica

- Este es un problema de modelado predictivo de clasificación con variables de entrada categóricas.

- La medida de correlación más común para los datos categóricos es la prueba de **chi cuadrado**.También podemos usar información mutua (ganancia de información) del campo de la teoría de la información.

- Las siguientes pruebas se pueden emplear en este caso:

    - Prueba de chi cuadrado (tablas de contingencia).
    - Información mutua.

        - De hecho, la información mutua es un método poderoso que puede resultar útil para datos categóricos y numéricos, p.Es agnóstico para los tipos de datos.

## **3. Creación de características**

✔️ Cuando tenermos pocas características o necesitamos mejorar el performance del modelo a través de nuevas variables.
 
1. Transformaciones matemáticas
2. Conteo
3. Fraccionar valores en sus partes (Ids, teléfonos, direcciones, etc)
4. Transformación de grupo

### Pautas:
- Los **modelos lineales** aprenden **sumas y restas** naturalmente, pero no pueden aprender nada más complejo.
- Las **proporciones parecen ser difíciles de aprender** para la mayoría de los modelos. 
    - Las **combinaciones de relación** a menudo conducen a algunas ganancias de rendimiento fáciles.
- Los **modelos lineales y las redes neuronales** generalmente funcionan mejor con **características normalizadas**. 
    - Las **redes neuronales** especialmente necesitan **características escaladas** a valores no muy lejos de 0. 
    - Los modelos basados ​​en **árboles** (como bosques aleatorios y xgboost) a veces pueden beneficiarse de la **normalización**, pero en general no.
- Los modelos de **árboles** pueden aprender a aproximar casi cualquier combinación de características, pero cuando una **combinación** es especialmente importante, aún se puede obtener beneficios al crearla explícitamente, especialmente cuando los datos son limitados.
- Los **recuentos son especialmente útiles para los modelos de árboles**, ya que estos modelos no tienen una forma natural de agregar información en muchas características a la vez.
